In [3]:
# 필요한 모듈 임포트
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate

# 데이터 읽어오기 (Q-질문 데이터, label-감정 라벨링)
train_file = "./chatbot_data.csv"
data = pd.read_csv(train_file, delimiter=',')
features = data['Q'].tolist()
labels = data['label'].tolist()

# 단어 인덱스 시퀀스 벡터 - 1)토큰으로 쪼개고 2)리스트로 만들고 3)순서마다 번호 부여
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
word_index = tokenizer.word_index
MAX_SEQ_LEN = 15  # 문장마다 다른 단어 시퀀스 벡터 크기를 동일하게 맞춤
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post') #패딩: max_seq_len보다 작은 벡터의 빈 공간에 0을 채움



# 학습용, 검증용, 테스트용 데이터셋 생성 ➌
# 학습셋:검증셋:테스트셋 = 7:2:1
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))
train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)
train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

# 하이퍼파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(word_index) + 1  # 전체 단어 수

# CNN 모델 정의
input_layer = Input(shape=(MAX_SEQ_LEN,)) #입력노드에 들어갈 shape로 시퀀스 벡터크기 지정
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer) #임베딩 계층 만들기; 시퀀스벡터를 희소벡터에서 밀집벡터로 변환
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer) #오버피팅에 대비함 

#크기가 각각 3,4,5인 필터 생성, 최대 풀링 연산 수행
conv1 = Conv1D(filters=128, kernel_size=3, padding='valid', activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)
conv2 = Conv1D(filters=128, kernel_size=4, padding='valid', activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)
conv3 = Conv1D(filters=128, kernel_size=5, padding='valid', activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

# 3, 4, 5- gram 이후 합치기: 3개의 Feature map 
concat = concatenate([pool1, pool2, pool3])
# 심층신경망. 128개의 출력노드와 relu 활성화함수
hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
# 예측할 값이 3개이므로 출력노드는 3개. 예측 최종 단계이므로 활성화함수 X 
logits = Dense(3, name='logits')(dropout_hidden)
# logits에서 나온 점수를 소프트맥스 계층으로 클래스 별 확률 계산  
predictions = Dense(3, activation=tf.nn.softmax)(logits)

# 모델 생성
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)

# 모델 평가(테스트 데이터셋 이용)
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Accuracy: %f' % (accuracy * 100))
print('loss: %f' % (loss))

# 모델 저장
model.save('cnn_model.h5')

Epoch 1/5
414/414 [==============================] - 24s 55ms/step - loss: 0.9080 - accuracy: 0.5466 - val_loss: 0.6677 - val_accuracy: 0.6569
Epoch 2/5
414/414 [==============================] - 23s 56ms/step - loss: 0.5442 - accuracy: 0.7815 - val_loss: 0.3027 - val_accuracy: 0.8997
Epoch 3/5
414/414 [==============================] - 23s 56ms/step - loss: 0.3249 - accuracy: 0.8891 - val_loss: 0.1453 - val_accuracy: 0.9518
Epoch 4/5
414/414 [==============================] - 23s 56ms/step - loss: 0.2031 - accuracy: 0.9340 - val_loss: 0.0862 - val_accuracy: 0.9746
Epoch 5/5
60/60 [==============================] - 0s 4ms/step - loss: 0.0841 - accuracy: 0.9763
Accuracy: 97.631133
loss: 0.084125
